In [662]:
allure_etudier=2

In [663]:
import  sys

#LOCALISATION DES DONNEES
sys.path.insert(0, "C:/projets_python/diabolo")

import warnings
warnings.filterwarnings("ignore")

from math import *


#LIBRAIRIES PERSO
import etude_variable.MyLog as log
import etude_variable.jouer as jouer

import etude_variable.analyse as ana

# LIBRAIRIE PYHTON CLASSIQUES
import pandas as pd
import numpy as np
import matplotlib
import scipy
import platform

from collections import Counter

#ESTIMATEUR
import sklearn
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn import neighbors, datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer

from sklearn.ensemble import GradientBoostingClassifier



from scipy.stats import uniform
from scipy.stats import randint

#TRAINING
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
import dask_searchcv as dcv
from skopt.space import Real, Integer
from sklearn.model_selection import cross_validate

#Evaluateur
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.feature_selection import SelectFromModel


from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.utils import class_weight

#outils
from dask.diagnostics import ProgressBar
import logging
from datetime import datetime


#Metriques
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn import  metrics
from sklearn.metrics import  roc_auc_score
from sklearn.metrics import  classification_report

#Outils

import time


#Graphique
from matplotlib import pyplot
from matplotlib import pyplot as plt
import seaborn as sns

print('Operating system version....', platform.platform())
print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)
print('scipy version is.......', scipy.__version__)


Operating system version.... Windows-10-10.0.17134-SP0
Python version is........... 3.6.5
scikit-learn version is..... 0.19.1
pandas version is........... 0.22.0
numpy version is............ 1.14.2
matplotlib version is....... 2.2.0
scipy version is....... 1.0.0


In [664]:
# Creating bins for the win column
def assign_selection(W):
    if W <=1:
        return 1
    else:
        return 0

In [665]:
# Creating bins for the win column
def assign_selection_cote(W,C):
    if W <=3 and C>4:
        return 1
    else:
        return 0

In [666]:
def lecture_data(Fichier, xnames, xindex_col, allure=1, mode_debug=0, avec_index=True):
    if avec_index==True:
        df = pd.read_csv(Fichier,  index_col=xindex_col,     sep=';',     names=xnames,               skipinitialspace=True,              encoding='Latin-1')
    else:
        df = pd.read_csv(Fichier,   index_col=None,  sep=';',                     names=xnames,                         skipinitialspace=True,     encoding='Latin-1')

    df = df.groupby("ALLURE")
    df = df.get_group(allure)
    #print(df.info())


    return df



In [667]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        print(start_time)
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        

        
def split_dataset(dataset, train_percentage, feature_headers,
                                  target_header,random_state=42,mode_debug=0):

    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size=train_percentage, test_size=None, random_state=42)


    if mode_debug==1:
        # Train and Test dataset size details
        print("--------------------------------")
        print("Train_x Shape :: ", train_x.shape)
        print("Train_y Shape :: ", train_y.shape)
        print("Test_x Shape :: ", test_x.shape)
        print("Test_y Shape :: ", test_y.shape)
        print("--------------------------------")

    return train_x, test_x, train_y, test_y


In [668]:
def my_drop(df, col):
    if col in df:
        df.drop([col], axis=1, inplace=True)
    return df

In [669]:
def suppression_colonne(df2,allure):
    # df=my_drop(df, "PAR_AGE")
    print('Suppression colonnne ', allure)

    if  allure==0:

        df2.drop(["FIN_ligne"], axis=1, inplace=True)
        df2.drop(["PAR_NP"], axis=1, inplace=True)
        df2.drop(["cendre"], axis=1, inplace=True)
        df2.drop(["MUSIC_CHEVAL"], axis=1, inplace=True)
        df2.drop(["MUSIC_ENT"], axis=1, inplace=True)
        df2.drop(["MUSIC_JOC"], axis=1, inplace=True)
        df2.drop(["grande_piste"], axis=1, inplace=True)
        df2.drop(["PAR_VALEUR"], axis=1, inplace=True)
        df2.drop(["autostart"], axis=1, inplace=True)
        #df2.drop(["CO_DISTANCE"], axis=1, inplace=True)
        #df2.drop(["Point"], axis=1, inplace=True)
        df2.drop(["CHEVAL"], axis=1, inplace=True)
        df2.drop(["HIPPO"], axis=1, inplace=True)
        df2.drop(["NOM_JOC"], axis=1, inplace=True)
        df2.drop(["NOM_ENTR"], axis=1, inplace=True)
        df2.drop(["CO_PRIX"], axis=1, inplace=True)
        df2.drop(["Point"], axis=1, inplace=True)

        df2.drop(["PAR_PROPRIO"], axis=1, inplace=True)
        df2.drop(["PAR_CARRIERE_Q"], axis=1, inplace=True)
        df2.drop(["PAR_PLACE_Q"], axis=1, inplace=True)
        df2.drop(["PAR_REUSSITE_QUINTE"], axis=1, inplace=True)
        df2.drop(["PAR_VICTOIRE_Q"], axis=1, inplace=True)
        df2.drop(["PAR_COTEDER"], axis=1, inplace=True)

        
    
        df2 = my_drop(df2, 'PAR_GAIN')     
        df2 = my_drop(df2, 'PAR_CLASSE_AGE')     


        
    if allure == 1:
        df2 = my_drop(df2, "POIDS")
        df2 = my_drop(df2, "CORDE")
   
    if allure==2:
        
        df2 = my_drop(df2, "MY_auto_start")
      #  df2 = my_drop(df2, "PAR_VICTOIRE_Q") 

    if allure == 3:
        df2 = my_drop(df2, "POIDS")
        df2 = my_drop(df2, "CORDE")
        df2 = my_drop(df2, "MY_auto_start")


    if allure == 4:
        df2 = my_drop(df2, "CORDE")
        df2 = my_drop(df2, "MY_auto_start")


    if allure == 5:
        df2 = my_drop(df2, "CORDE")
        df2 = my_drop(df2, "MY_auto_start")


    return df2

In [670]:
  # Lecture des données sans index
xnames = ['ALLURE', 'CO_DISTANCE',
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE', 'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant',
              'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'CORDE', 'CHEVAL',
              'MUSIC_CHEVAL',
              'MUSIC_ENT',
              'MUSIC_JOC',
              'PAR_VALEUR',
              'PAR_ENT_ECART_PLACE',
              'PAR_VICTOIRE',
              'PAR_VICTOIRE_Q',
          'PAR_ENT_NB_COURSE',
              'FIN_ligne']


xnames2 = [
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant',
              'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'CORDE', 'CHEVAL',
              'MUSIC_CHEVAL',
              'MUSIC_ENT',
              'MUSIC_JOC',
              'PAR_VALEUR',
              'PAR_ENT_ECART_PLACE',
              'PAR_VICTOIRE',
              'PAR_VICTOIRE_Q',
              'PAR_ENT_NB_COURSE',
              'FIN_ligne',
               'SELECTION2',
               'p2018']
    

In [671]:
def transformation(df2,allure_etudier):
    #Filtre des données¶
    print('\n\n----------- TRANSFORMATION --------- AJOUT DE COLONNE------------\n')
    start_time=timer()
    #df2=df2[df2.Point>0]
    df2 = df2[df2.PAR_COTEDER > 0]
   # df2 = df2[df2.CO_PRIX > 0]
    df2 = df2[df2.PAR_GAIN >= 0]
    df2 = df2[df2.Nb_partant >= 7]
          #  df2.drop(["Nb_partant"], axis=1, inplace=True)
    df2 = df2[df2.PAR_GAIN < 1000000]
    df2 = df2[df2.PAR_ARRIVE > 0]  # On garde la ligne qui possede information arrivée
    
    df2 = df2.groupby("ALLURE")
    df2 = df2.get_group(allure_etudier)
    df2 = my_drop(df2, 'ALLURE')
    
    if allure_etudier == 2 or allure_etudier == 4 or allure_etudier == 5:
        df2 = df2[df2.POIDS > 20]
        df2 = df2[df2.POIDS < 80]
        
    #  on a lu les données avant l 'ajout de la colonne SELECTIOB
    df2['SELECTION2'] = df2['PAR_ARRIVE'].apply(assign_selection)
    #df2['SELECTION2'] = df2.apply(lambda x: assign_selection_cote(x['PAR_ARRIVE'], x['PAR_COTEDER']), axis=1)    
    
    
    
    
    df2 = my_drop(df2, 'PAR_ARRIVE')

    #df2['CHEVAL_QUINTE'] = df2.apply(lambda x: CHEVAL_QUINTE(x['PAR_CARRIERE_Q']), axis=1)    
    timer(start_time)
    return df2
   



In [672]:

def somme_note(df,num_course,cri):
   
    df.loc[df['IDCOURSE']==num_course,'p2018']  = df.loc[df['IDCOURSE']==num_course,'p2018'] +   df.loc[df['IDCOURSE']==num_course, cri]
    
    
    return df

        
     
            
def calcul_note(df, num_course, cri):
    df.loc[df['IDCOURSE']==num_course,'aux'] = df.loc[df['IDCOURSE']==num_course,cri[0]]                                                
    valeur_critere=df.loc[df['IDCOURSE']==num_course,cri]  # selection critere     
    maxx=valeur_critere.max() #MAXI    
    
    if ((maxx[0])>0):
        df.loc[df['IDCOURSE']==num_course,cri ]= ( df.loc[df['IDCOURSE']==num_course,cri]  /maxx[0]) *20  
    else:
        df.loc[df['IDCOURSE']==num_course,cri]=0
                        
    df=somme_note(df,num_course, cri[0])
    df.loc[df['IDCOURSE']==num_course,cri[0]] =    df.loc[df['IDCOURSE']==num_course,'aux']            
    
    return  df


#-------------------------------------------------------------------------------------------------------------------------------------------------------
def calcul_les_notes(df):
    i=0
    start_time=timer()
    df['p2018']=0
    df['aux']=0

        
    
    for n  in df.groupby(['IDCOURSE'],axis=0):
        nc=n[0]
        if math.fmod(i,500)==0 or i<20:
            print("    print(N° ",i,timer(start_time))
            #df= calcul_note(df, nc, ['PAR_CARRIERE'] )
        df= calcul_note(df, nc, ['PAR_ENT_ECART_GAGNANT'] )
        df= calcul_note(df, nc, ['PAR_ENT_RAPPORT_GAGNANT_M'] )
        df= calcul_note(df, nc, ['PAR_ENT_REU_PLACE'] )
        df= calcul_note(df, nc, ['PAR_ENT_REUSSITE_GAGNE'] )
        df= calcul_note(df, nc, ['PAR_ENT_VICTOIRE'] ) 
        df= calcul_note(df, nc, ['PAR_ENT_NB_COURSE'] )   
    
        df= calcul_note(df, nc, ['pAR_JOC_ECART_GAGNANT'] )
        df= calcul_note(df, nc, ['pAR_JOC_RAPPORT_GAGNANT_M'] )
        df= calcul_note(df, nc, ['PAR_JOC_REU_PLACE'] )
        
        df= calcul_note(df, nc, ['pAR_JOC_REUSSITE_GAGNE'] )
        df= calcul_note(df, nc, ['pAR_JOC_VICTOIRE'] )    
        df= calcul_note(df, nc, ['PAR_JOC_ECART_PLACE'] )
        df= calcul_note(df, nc, ['PAR_JOC_NB_COURSE'] )    
        df= calcul_note(df, nc, ['PAR_JOC_PLACE_3P'] )    
    
        df= calcul_note(df, nc, ['PAR_REUSSITE_GAGNE'] )
        df= calcul_note(df, nc, ['PAR_RUESSITE_PLACE'] )
        df= calcul_note(df, nc, ['PAR_REUSSITE_3P'] )    
        df= calcul_note(df, nc, ['PAR_CARRIERE'] )
        df= calcul_note(df, nc, ['PAR_CARRIERE_Q'] )
        df= calcul_note(df, nc, ['PAR_GAIN'] )
        df= calcul_note(df, nc, ['PAR_PLACE'] )
        df= calcul_note(df, nc, ['PAR_PLACE_Q'] )
        df= calcul_note(df, nc, ['PAR_CLASSE_AGE'] )
        df= calcul_note(df, nc, ['PAR_VICTOIRE'] )
        df= calcul_note(df, nc, ['PAR_REUSSITE_QUINTE'] )
        df= calcul_note(df, nc, ['PAR_VICTOIRE_Q'] )
        df= calcul_note(df, nc, ['PAR_ENT_ECART_PLACE'] )
      
        i=i+1
   
            

    return df

In [673]:
def nettoyer_Nan(df):
    df[['PAR_ENT_ECART_GAGNANT']] = df[['PAR_ENT_ECART_GAGNANT']].fillna( df[['PAR_ENT_ECART_GAGNANT']].mean())
    df[['PAR_ENT_RAPPORT_GAGNANT_M']] = df[['PAR_ENT_RAPPORT_GAGNANT_M']].fillna( df[['PAR_ENT_RAPPORT_GAGNANT_M']].mean())
    df[['PAR_ENT_REU_PLACE']] = df[['PAR_ENT_REU_PLACE']].fillna( df[['PAR_ENT_REU_PLACE']].mean())
    df[['PAR_ENT_REUSSITE_GAGNE']] = df[['PAR_ENT_REUSSITE_GAGNE']].fillna( df[['PAR_ENT_REUSSITE_GAGNE']].mean())
    df[['PAR_ENT_VICTOIRE']] = df[['PAR_ENT_VICTOIRE']].fillna( df[['PAR_ENT_VICTOIRE']].mean())
    df[['PAR_ENT_NB_COURSE']] = df[['PAR_ENT_NB_COURSE']].fillna( df[['PAR_ENT_NB_COURSE']].mean())
    
    df[['pAR_JOC_ECART_GAGNANT']] = df[['pAR_JOC_ECART_GAGNANT']].fillna( df[['pAR_JOC_ECART_GAGNANT']].mean())
    df[['pAR_JOC_RAPPORT_GAGNANT_M']] = df[['pAR_JOC_RAPPORT_GAGNANT_M']].fillna( df[['pAR_JOC_RAPPORT_GAGNANT_M']].mean())
    df[['PAR_JOC_REU_PLACE']] = df[['PAR_JOC_REU_PLACE']].fillna( df[['PAR_JOC_REU_PLACE']].mean())
    df[['pAR_JOC_REUSSITE_GAGNE']] = df[['pAR_JOC_REUSSITE_GAGNE']].fillna( df[['pAR_JOC_REUSSITE_GAGNE']].mean())
    df[['pAR_JOC_VICTOIRE']] = df[['pAR_JOC_VICTOIRE']].fillna( df[['pAR_JOC_VICTOIRE']].mean())
    df[['PAR_JOC_ECART_PLACE']] = df[['PAR_JOC_ECART_PLACE']].fillna( df[['PAR_JOC_ECART_PLACE']].mean())
    df[['PAR_JOC_NB_COURSE']] = df[['PAR_JOC_NB_COURSE']].fillna( df[['PAR_JOC_NB_COURSE']].mean())
    df[['PAR_JOC_PLACE_3P']] = df[['PAR_JOC_PLACE_3P']].fillna( df[['PAR_JOC_PLACE_3P']].mean())
    df['PAR_RUESSITE_PLACE'] = df['PAR_RUESSITE_PLACE'].fillna( df['PAR_RUESSITE_PLACE'].mean())
    df[['PAR_REUSSITE_GAGNE']] = df[['PAR_REUSSITE_GAGNE']].fillna( 0)
    df[['PAR_REUSSITE_3P']] = df[['PAR_REUSSITE_3P']].fillna( df[['PAR_REUSSITE_3P']].mean())
    df[['PAR_CARRIERE']] = df[['PAR_CARRIERE']].fillna( df[['PAR_CARRIERE']].mean())
    df[['PAR_CARRIERE_Q']] = df[['PAR_CARRIERE_Q']].fillna( df[['PAR_CARRIERE_Q']].mean())
    df[['PAR_GAIN']] = df[['PAR_GAIN']].fillna( df[['PAR_GAIN']].mean())    
    df[['PAR_PLACE']] = df[['PAR_PLACE']].fillna( df[['PAR_PLACE']].mean())
    df[['PAR_PLACE_Q']] = df[['PAR_PLACE_Q']].fillna( df[['PAR_PLACE_Q']].mean())
    df[['PAR_CLASSE_AGE']] = df[['PAR_CLASSE_AGE']].fillna( df[['PAR_CLASSE_AGE']].mean())
    df[['PAR_POINT']] = df[['PAR_POINT']].fillna( df[['PAR_POINT']].mean())
    df[['PAR_REUSSITE_QUINTE']] = df[['PAR_REUSSITE_QUINTE']].fillna( df[['PAR_REUSSITE_QUINTE']].mean())
    df[['PAR_VICTOIRE_Q']] = df[['PAR_VICTOIRE_Q']].fillna( df[['PAR_VICTOIRE_Q']].mean())
    df[['PAR_ENT_ECART_PLACE']] = df[['PAR_ENT_ECART_PLACE']].fillna( df[['PAR_ENT_ECART_PLACE']].mean())
        
    return df


import math
def relation_1(a,b):
    if b==0:
        return 0
    else:
        return (a/b)*20
    


In [674]:
index_col = ['IDPARTCIPANT', 'IDCOURSE']
start_time=timer()
df = lecture_data('d:\data_diabolo.csv', xnames, xindex_col=index_col, allure=allure_etudier, avec_index=False)
df = df.groupby("ALLURE")
df = df.get_group(allure_etudier)
timer(start_time)


2018-07-10 22:28:33.967763

 Time taken: 0 hours 0 minutes and 3.18 seconds.


In [675]:
df[['Nb_partant']].head(10)

,Nb_partant
100,14
101,14
102,14
103,14
104,14
105,14
106,14
107,14
108,14
109,14


In [676]:
df2=transformation(df,allure_etudier)

    





----------- TRANSFORMATION --------- AJOUT DE COLONNE------------

2018-07-10 22:28:38.546104

 Time taken: 0 hours 0 minutes and 0.44 seconds.


In [ ]:
df2=calcul_les_notes(df2)

2018-07-10 22:28:40.835960

 Time taken: 0 hours 0 minutes and 0.09 seconds.
    print(N°  0 None

 Time taken: 0 hours 0 minutes and 1.27 seconds.
    print(N°  1 None

 Time taken: 0 hours 0 minutes and 1.78 seconds.
    print(N°  2 None

 Time taken: 0 hours 0 minutes and 2.28 seconds.
    print(N°  3 None

 Time taken: 0 hours 0 minutes and 2.76 seconds.
    print(N°  4 None

 Time taken: 0 hours 0 minutes and 3.31 seconds.
    print(N°  5 None

 Time taken: 0 hours 0 minutes and 3.85 seconds.
    print(N°  6 None

 Time taken: 0 hours 0 minutes and 4.33 seconds.
    print(N°  7 None

 Time taken: 0 hours 0 minutes and 4.84 seconds.
    print(N°  8 None

 Time taken: 0 hours 0 minutes and 5.4 seconds.
    print(N°  9 None

 Time taken: 0 hours 0 minutes and 5.91 seconds.
    print(N°  10 None

 Time taken: 0 hours 0 minutes and 6.4 seconds.
    print(N°  11 None

 Time taken: 0 hours 0 minutes and 6.9 seconds.
    print(N°  12 None

 Time taken: 0 hours 0 minutes and 7.38 seconds.


In [ ]:

df2


In [ ]:
DF3 =df2.copy()


In [ ]:
df2
df2=DF3.copy()

In [ ]:
DF3.head()

In [ ]:
  


if allure_etudier==1:
    df2.to_csv('d:\diabolo_1_note.csv',sep=';',columns =xnames2, header=False)
if allure_etudier==2:
    Df2.to_csv('d:\diabolo_2_note.csv',sep=';',columns =xnames2, header=False)    
if allure_etudier==3:
    df2.to_csv('d:\diabolo_3_note.csv',sep=';',columns =xnames2, header=False)    
    



In [ ]:
print(xnames2)


In [ ]:
if allure_etudier==1:
    df2=pd.read_csv('d:\diabolo_1_note.csv',   sep=';',  names=xnames2,                         skipinitialspace=True,     encoding='utf-8')
    df2=df2.drop(df2.index[0])
    df2.head(10)
    
if allure_etudier==2:
    df2=pd.read_csv('d:\diabolo_2_note.csv',   sep=';',  names=xnames2,                         skipinitialspace=True,     encoding='utf-8')
    df2=df2.drop(df2.index[0])
    df2.head(10)
    
if allure_etudier==3:
    df2=pd.read_csv('d:\diabolo_3_note.csv',   sep=';',  names=xnames2,                         skipinitialspace=True,     encoding='utf-8')
    df2=df2.drop(df2.index[0])
    df2.head(10)    

In [ ]:
df2.head(10)


In [ ]:
def conversion(df2):
    df2['CO_PRIX']=df2['CO_PRIX'].astype('int')
    df2['HIPPO']=df2['HIPPO'].astype('int')
    
    df2['IDCOURSE']=df2['IDCOURSE'].astype('int')
    df2['IDPARTCIPANT']=df2['IDPARTCIPANT'].astype('int')
    df2['PAR_AGE']=df2['PAR_AGE'].astype('int')
    df2['PAR_CARRIERE']=df2['PAR_CARRIERE'].astype('float')
    df2['PAR_CARRIERE']=df2['PAR_CARRIERE'].astype('int')
    df2['PAR_CARRIERE_Q']=df2['PAR_CARRIERE_Q'].astype('float')
    df2['PAR_CARRIERE_Q']=df2['PAR_CARRIERE_Q'].astype('int')
    df2['PAR_CLASSE_AGE']=df2['PAR_CLASSE_AGE'].astype('float')
    df2['PAR_CLASSE_AGE']=df2['PAR_CLASSE_AGE'].astype('int')
    df2['PAR_COTEDER']=df2['PAR_COTEDER'].astype('float')
    df2['PAR_ENT_ECART_GAGNANT']=df2['PAR_ENT_ECART_GAGNANT'].astype('float')
    df2['PAR_ENT_ECART_GAGNANT']=df2['PAR_ENT_ECART_GAGNANT'].astype('int')
    df2['PAR_ENT_RAPPORT_GAGNANT_M']=df2['PAR_ENT_RAPPORT_GAGNANT_M'].astype('float')
    df2['PAR_ENT_RAPPORT_GAGNANT_M']=df2['PAR_ENT_RAPPORT_GAGNANT_M'].astype('int')
    df2['PAR_ENT_REU_PLACE']=df2['PAR_ENT_REU_PLACE'].astype('float')
    df2['PAR_ENT_REUSSITE_GAGNE']=df2['PAR_ENT_REUSSITE_GAGNE'].astype('float')
    df2['PAR_ENT_VICTOIRE']=df2['PAR_ENT_VICTOIRE'].astype('float')
    df2['PAR_ENT_VICTOIRE']=df2['PAR_ENT_VICTOIRE'].astype('int')
    df2['PAR_GAIN']=df2['PAR_GAIN'].astype('float')
    df2['PAR_GAIN']=df2['PAR_GAIN'].astype('int')
    df2['pAR_JOC_ECART_GAGNANT']=df2['pAR_JOC_ECART_GAGNANT'].astype('float')
    df2['pAR_JOC_ECART_GAGNANT']=df2['pAR_JOC_ECART_GAGNANT'].astype('int')
    df2['PAR_JOC_ECART_PLACE']=df2['PAR_JOC_ECART_PLACE'].astype('float')
    df2['PAR_JOC_ECART_PLACE']=df2['PAR_JOC_ECART_PLACE'].astype('int')
    df2['PAR_JOC_NB_COURSE']=df2['PAR_JOC_NB_COURSE'].astype('float')
    df2['PAR_JOC_NB_COURSE']=df2['PAR_JOC_NB_COURSE'].astype('int')
    df2['PAR_JOC_PLACE_3P']=df2['PAR_JOC_PLACE_3P'].astype('float')
    df2['PAR_JOC_PLACE_3P']=df2['PAR_JOC_PLACE_3P'].astype('int')
    df2['pAR_JOC_RAPPORT_GAGNANT_M']=df2['pAR_JOC_RAPPORT_GAGNANT_M'].astype('float')
    df2['pAR_JOC_RAPPORT_GAGNANT_M']=df2['pAR_JOC_RAPPORT_GAGNANT_M'].astype('int')
    df2['PAR_JOC_REU_PLACE']=df2['PAR_JOC_REU_PLACE'].astype('float')
    df2['pAR_JOC_REUSSITE_GAGNE']=df2['pAR_JOC_REUSSITE_GAGNE'].astype('float')
    df2['pAR_JOC_VICTOIRE']=df2['pAR_JOC_VICTOIRE'].astype('float')
    df2['pAR_JOC_VICTOIRE']=df2['pAR_JOC_VICTOIRE'].astype('int')
    df2['PAR_NP']=df2['PAR_NP'].astype('float')
    df2['PAR_NP']=df2['PAR_NP'].astype('int')
    df2['PAR_NUM']=df2['PAR_NUM'].astype('float')
    df2['PAR_NUM']=df2['PAR_NUM'].astype('int')
    df2['PAR_PLACE']=df2['PAR_PLACE'].astype('float')
    df2['PAR_PLACE']=df2['PAR_PLACE'].astype('int')
    df2['PAR_PLACE_Q']=df2['PAR_PLACE_Q'].astype('float')
    df2['PAR_PLACE_Q']=df2['PAR_PLACE_Q'].astype('int')
    df2['PAR_REUSSITE_3P']=df2['PAR_REUSSITE_3P'].astype('float')
    df2['PAR_REUSSITE_GAGNE']=df2['PAR_REUSSITE_GAGNE'].astype('float')
    df2['PAR_REUSSITE_QUINTE']=df2['PAR_REUSSITE_QUINTE'].astype('float')
    df2['PAR_RUESSITE_PLACE']=df2['PAR_RUESSITE_PLACE'].astype('float')
    df2['autostart']=df2['autostart'].astype('float')
    df2['autostart']=df2['autostart'].astype('int')
    df2['cendre']=df2['cendre'].astype('float')
    df2['cendre']=df2['cendre'].astype('int')
    df2['grande_piste']=df2['grande_piste'].astype('float')
    df2['grande_piste']=df2['grande_piste'].astype('int')
    df2['Point']=df2['Point'].astype('float')
    df2['Point']=df2['Point'].astype('int')
    df2['Nb_partant']=df2['Nb_partant'].astype('float')
    df2['Nb_partant']=df2['Nb_partant'].astype('int')
    df2['PAR_PROPRIO']=df2['PAR_PROPRIO'].astype('float')
    df2['PAR_PROPRIO']=df2['PAR_PROPRIO'].astype('int')
    df2['NOM_JOC']=df2['NOM_JOC'].astype('float')
    df2['NOM_JOC']=df2['NOM_JOC'].astype('int')
    df2['NOM_ENTR']=df2['NOM_ENTR'].astype('float')
    df2['NOM_ENTR']=df2['NOM_ENTR'].astype('int')
    df2['POIDS']=df2['POIDS'].astype('float')
    df2['POIDS']=df2['POIDS'].astype('int')
    df2['CORDE']=df2['CORDE'].astype('float')
    df2['CORDE']=df2['CORDE'].astype('int')
    df2['CHEVAL']=df2['CHEVAL'].astype('float')
    df2['CHEVAL']=df2['CHEVAL'].astype('int')
    df2['MUSIC_CHEVAL']=df2['MUSIC_CHEVAL'].astype('str')
    df2['MUSIC_ENT']=df2['MUSIC_ENT'].astype('str')
    df2['MUSIC_JOC']=df2['MUSIC_JOC'].astype('str')
    df2['PAR_VALEUR']=df2['PAR_VALEUR'].astype('float')
    df2['PAR_VALEUR']=df2['PAR_VALEUR'].astype('int')
    df2['PAR_ENT_ECART_PLACE']=df2['PAR_ENT_ECART_PLACE'].astype('float')
    df2['PAR_ENT_ECART_PLACE']=df2['PAR_ENT_ECART_PLACE'].astype('int')
    df2['PAR_VICTOIRE']=df2['PAR_VICTOIRE'].astype('float')
    df2['PAR_VICTOIRE']=df2['PAR_VICTOIRE'].astype('int')
    df2['PAR_VICTOIRE_Q']=df2['PAR_VICTOIRE_Q'].astype('float')
    df2['PAR_VICTOIRE_Q']=df2['PAR_VICTOIRE_Q'].astype('int')
    df2['PAR_ENT_NB_COURSE']=df2['PAR_ENT_NB_COURSE'].astype('float')
    df2['PAR_ENT_NB_COURSE']=df2['PAR_ENT_NB_COURSE'].astype('int')
    df2['FIN_ligne']=df2['FIN_ligne'].astype('str')

    df2['p2018']=df2['p2018'].astype('float')
    df2['p2018']=df2['p2018'].astype('int')
    df2['PAR_REUSSITE_QUINTE']=df2['PAR_REUSSITE_QUINTE'].astype('float')
    df2['PAR_VICTOIRE_Q']=df2['PAR_VICTOIRE_Q'].astype('float')
    df2['PAR_VICTOIRE_Q']=df2['PAR_VICTOIRE_Q'].astype('int')
    
    return df2


    

In [ ]:
df2=conversion(df2)
df2['SELECTION2']=df2['SELECTION2'].astype('float')
df2['SELECTION2']=df2['SELECTION2'].astype('int')
                

In [ ]:
df2.info()

In [ ]:
df2[['IDPARTCIPANT','PAR_REUSSITE_3P']].head(10)

In [ ]:
#df2=transformationPoint(df2,allure_etudier)
#df2, critere_scale = encodage(df2)
df2.head(10)

In [ ]:
df2 = suppression_colonne(df2=df2,allure=0)
df2 = suppression_colonne(df2=df2,allure=allure_etudier)


In [ ]:
df[['IDPARTCIPANT','PAR_REUSSITE_3P']].head(10)

In [ ]:
#df2[['pAR_JOC_REUSSITE_GAGNE']].describe()



# Definition de la cible

In [ ]:
# Creating bins for the win column
def assign_selection2(W):
    if W==1:
        return 1
    if W==0:
        return 0
    
    

Lib_features_df = df2.columns
xdata = df2.values

df_gagnant = pd.DataFrame(data=xdata, columns=Lib_features_df)
df_gagnant['SELECTION'] = df_gagnant['SELECTION2'].apply(assign_selection2)
df_gagnant.drop(["SELECTION2"], axis=1, inplace=True)

df_gagnant = df_gagnant.set_index(index_col)

df_gagnant_len = len(df_gagnant.columns) - 1
Lib_features = df_gagnant.columns[:df_gagnant_len]

feature_columns = Lib_features  ##<<<<<<<<<<<<<<<<
response_column = ['SELECTION']  ##<<<<<<<<<<<<<<<<
log.traceLogdebug("Features                   : %s " % Lib_features, " <<<<************")

print("(1) Shape df_gagnant  ", df_gagnant.shape,"\n")
print("(2) FEATURES ",Lib_features)
print("(3) response column ",response_column)
# df_gagnant, feature_columns, response_column, ratio SONT DEFINI


# Suppression de colonnes

In [ ]:
#df_gagnant_len = len(df_gagnant.columns) - 1
#Lib_features = df_gagnant.columns[:df_gagnant_len]
feature_columns = Lib_features  ##<<<<<<<<<<<<<<<<
#response_column = ['SELECTION']  ##<<<<<<<<<<<<<<<<
log.traceLogdebug("Features                   : %s " % Lib_features, " <<<<************")

print("(1) Shape df_gagnant  ", df_gagnant.shape,"\n")
print("(2) FEATURES\n ",Lib_features,"\n")
print("(3) response column ",response_column,"\n")
df_gagnant.info()
    

# CORRELATION


In [ ]:
#Enfin, quelque chose de vraiment cool que vous pouvez mettre 

#dans une carte de chaleur est une matrice de corrélation. 
#Pandas DataFrame a une méthode corr qui calcule le coefficient de corrélation de Pearson (peut être un autre)
#entre tous les couples de colonnes numériques du DataFrame.

fig, ax = plt.subplots(figsize=(22,22))
#sns.set(font_scale=6)
sns.heatmap(df_gagnant.corr(), annot=True, fmt=".2f", linewidths=.0, ax=ax, annot_kws={"size": 12}, xticklabels = 1 )


In [ ]:
target_count = df_gagnant['SELECTION'].value_counts()


print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')
target_count.plot(kind='bar', title='Count (target)')

normal_trans_perc = sum(df_gagnant['SELECTION'] == 0) / (sum(df_gagnant['SELECTION'] == 0) + sum(df_gagnant['SELECTION'] == 1))
fraud_trans_perc = 1 - normal_trans_perc
print('Total number of records : {} '.format(len(df_gagnant)))
print('Nombre de participations avec SELECTION = 0 : {}'.format(sum(df_gagnant['SELECTION'] == 0)))
print('Nombre de participations avec SELECTION = 1  : {}'.format(sum(df_gagnant['SELECTION'] == 1)))
print('Pourcentage 0: {:.4f}%,  pourcentage 1 : {:.4f}%'.format(normal_trans_perc * 100,fraud_trans_perc * 100))




# Definition des set

In [ ]:
def courbe_de_roc(model, test_x,test_y):
    from sklearn.metrics import roc_curve, auc

    probas = model.predict_proba(test_x)
    # probas est une matrice de deux colonnes avec la proabilités d'appartenance à chaque classe

    fpr, tpr, thresholds = roc_curve(test_y, probas[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("Area under the ROC curve : %f" % roc_auc)
        
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")   
    
def plot_importance(feature_columns, model):
    importances = pd.DataFrame({'feature': feature_columns, 'importance': np.round(model.feature_importances_, 3)})
    importances = importances.sort_values('importance', ascending=False).set_index('feature')
    print("")
    print( importances)
    importances.plot.bar()
    
    
def metrique_classe(y_pred,y_true,xclass):
    from imblearn.metrics import specificity_score
    from imblearn.metrics import sensitivity_score


    from imblearn.metrics import geometric_mean_score



    # La sensibilité est le rapport où est le nombre de vrais positifs et le nombre de faux négatifs.
    # La sensibilité quantifie la capacité à éviter les faux négatifs.tp


    # estimator issu de quelques FIT




    log.traceLogInfo("Sensibilité  du re-equilibrage des données sur le TEST")
    #log.traceLogInfo("Binary ",sensitivity_score(y_true, y_pred, average='binary', pos_label=xclass))

    log.traceLogInfo("La spécificité est intuitivement la capacité du classificateur à trouver tous les échantillons positifs")

    log.traceLogInfo("Binary ")
    log.traceLogInfo(specificity_score(y_true, y_pred, labels=None, pos_label=xclass, average='binary', sample_weight=None))




    print("\nCalculer la moyenne géométrique")
    print(geometric_mean_score(y_true, y_pred,labels=None, pos_label=xclass))

    print("\n Calculer  sensitivity score")
    print("La sensibilité est le rapport où est le nombre de vrais positifs et le nombre de faux négatifs.")
    print("La sensibilité quantifie la capacité à éviter les faux négatifs.")

    print(sensitivity_score(y_true, y_pred, labels=None, pos_label=xclass,average='binary'))

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df_gagnant[feature_columns],
                                                     df_gagnant[response_column], test_size=0.25, random_state=42)
train_eval_X =  df_gagnant[feature_columns]
train_eval_Y=  df_gagnant[response_column]





print("Train_x",train_x.shape)
print("Train y",train_y.shape)
print("Test x",test_x.shape)
print("Test y",test_y.shape)

print("train_eval_X",train_eval_X.shape)
print("train_eval_Y",train_eval_Y.shape)

print("Test x",test_x.shape)
print("Test y",test_y.shape)

test_y = test_y['SELECTION'].ravel()
train_y = train_y['SELECTION'].ravel()



def smot2(train_x,train_y,feature_columns):
    start_time=timer()
    from imblearn.combine import SMOTEENN
    from imblearn.over_sampling import SMOTE


    #print('\nOriginal dataset shape {}'.format(Counter(train_y)))
    sm = SMOTEENN(ratio='minority',n_jobs=3,random_state=42,
                         n_neighbors=15,smote=SMOTE())
    
    sm = SMOTE(ratio='minority', n_jobs=3, random_state=42,m_neighbors=5)


    X_res, y_res = sm.fit_sample(train_x, train_y)



    train_x = pd.DataFrame(X_res, columns=feature_columns)
    train_y = pd.Series(y_res)
    print("Fin SMOT")
    timer(start_time)


    return train_x,train_y





In [ ]:
silence_esti=False
#train_x, train_y = smot2(train_x=train_x, train_y=train_y, feature_columns=feature_columns)
#test_x, test_y = smot2(train_x=test_x, train_y=test_y, feature_columns=feature_columns)


In [ ]:
if allure_etudier == 1:
                  estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.6,
       colsample_bytree=0.7, gamma=0.0, learning_rate=0.04,
       max_delta_step=3, max_depth=11, max_features='sqrt',
       min_child_weight=6, n_estimators=1800, n_jobs=2,
       nthread=None, objective='binary:logistic', random_state=42,
       reg_alpha=1e-05, reg_lambda=5.0, scale_pos_weight=1, seed=80,
       silent=False, subsample=1)
if allure_etudier == 3:
                estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=0.9, gamma=0.3, learning_rate=0.07,
                max_delta_step=4, max_depth=10, max_features='sqrt',
                min_child_weight=13, n_estimators=1800, n_jobs=1,
                nthread=None, objective='binary:logistic', random_state=10,
                reg_alpha=0.2, reg_lambda=1, scale_pos_weight=1, seed=400,
                silent=True, subsample=0.9)

if allure_etudier == 2:
                estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0.2, learning_rate=0.07,
       max_delta_step=8, max_depth=7, max_features='sqrt',
       min_child_weight=2, n_estimators=2430, n_jobs=2,
       nthread=None, objective='binary:logistic', random_state=10,
       reg_alpha=1e-05, reg_lambda=3, scale_pos_weight=6, seed=1090,
       silent=False, subsample=1)

if allure_etudier == 4:
                 estimator =  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=0.6, gamma=0.1, learning_rate=0.07,
                max_delta_step=0, max_depth=9, max_features='sqrt',
                min_child_weight=14, n_estimators=100, n_jobs=1,
                nthread=None, objective='binary:logistic', random_state=10,
                reg_alpha=0.03, reg_lambda=1, scale_pos_weight=1, seed=27,
                silent=True, subsample=0.9)

if allure_etudier == 5:
                estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                       colsample_bytree=0.6, gamma=0.1, learning_rate=0.07,
                       max_delta_step=0, max_depth=9, max_features='sqrt',
                       min_child_weight=12, n_estimators=100, n_jobs=1,
                       nthread=None, objective='binary:logistic', random_state=10,
                       reg_alpha=0.03, reg_lambda=1, scale_pos_weight=1, seed=27,
                       silent=True, subsample=0.9)


        
model=estimator


In [ ]:
start_time=timer()
eval_set = [(train_x, train_y), (test_x, test_y)]
model.fit(train_x, train_y, eval_metric=["error", "auc"], eval_set=eval_set, verbose=False, early_stopping_rounds=75)

print("***********")
timer(start_time)          

In [ ]:
y_pred = model.predict(test_x)
predictions = [round(value) for value in y_pred]
PROBA = model.predict_proba(test_x)

In [ ]:
 # evaluate predictions
accuracy = accuracy_score(test_y, predictions)
print(">>>>>>>>>>  Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
print(classification_report(test_y, predictions))

In [ ]:
plot_importance(feature_columns=feature_columns,model=model)

In [ ]:
df_numero_a_predire = lecture_data('d:\data_jour.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure_etudier,avec_index=False)
print("Fichier lu d:\data_jour.csv" )

df_numero_a_predire = df_numero_a_predire.groupby("ALLURE")
df_numero_a_predire = df_numero_a_predire.get_group(allure_etudier)



In [ ]:
df2_journee=calcul_les_notes(df_numero_a_predire)

In [ ]:

df2_journee=conversion(df2_journee)

In [ ]:
#df2=transformation(df_numero_a_predire,allure_etudier)

df2_journee = suppression_colonne(df2=df2_journee,allure=0)
df2_journee = suppression_colonne(df2=df2_journee,allure=allure_etudier)




#df_numero_a_predire=transformation_courses(df_numero_a_predire,allure_etudier)

#df_numero_a_predire, critere_scale =encodage(df_numero_a_predire)



#df_numero_a_predire =suppression_colonne(df_numero_a_predire, allure=0)
#df_numero_a_predire =suppression_colonne(df_numero_a_predire, allure=allure_etudier)

df2_journee['SELECTION'] = 0


#print(df_numero_a_predire.info())
print('\n')
print(feature_columns)
print(" \n ------SUPPRESSIOOn OK------------")


In [ ]:
df2_journee=df2_journee.set_index( ['IDPARTCIPANT', 'IDCOURSE'])

In [ ]:
df2_journee.head(10)

In [ ]:
train_x, test_x, train_y, test_y = split_dataset(df2_journee, 0, feature_columns, response_column)

test_y = test_y['SELECTION'].ravel()


y_pred = model.predict(test_x)
y_pred

In [ ]:
df_pred = pd.DataFrame.from_dict(y_pred)

test_copy = test_x.copy() #################

PROBA = model.predict_proba(test_x)


In [ ]:
df_proba = pd.DataFrame.from_dict(PROBA)    
df_final = pd.concat([df_proba, df_pred], axis=1)
df_final.head(10)

In [ ]:

test_x = test_copy.copy()

test_x =my_drop(test_x, 'PAR_CLASSE_AGE')

test_x =my_drop(test_x, 'PAR_ENT_ECART_GAGNANT')
test_x =my_drop(test_x, 'PAR_ENT_RAPPORT_GAGNANT_M')
test_x =my_drop(test_x, 'PAR_ENT_REU_PLACE')
test_x =my_drop(test_x, 'PAR_ENT_REUSSITE_GAGNE')
test_x =my_drop(test_x, 'PAR_ENT_VICTOIRE')
test_x =my_drop(test_x, 'PAR_GAIN')
test_x =my_drop(test_x, 'pAR_JOC_ECART_GAGNANT')
test_x =my_drop(test_x, 'PAR_JOC_ECART_PLACE')
test_x =my_drop(test_x, 'PAR_JOC_NB_COURSE')
test_x =my_drop(test_x, 'PAR_JOC_PLACE_3P')
test_x =my_drop(test_x, 'PAR_REUSSITE_GAGNE')
test_x =my_drop(test_x, 'PAR_REUSSITE_QUINTE')
test_x =my_drop(test_x, 'PAR_RUESSITE_PLACE')
test_x =my_drop(test_x, 'CO_PRIX')
test_x =my_drop(test_x, 'PAR_CARRIERE')
test_x =my_drop(test_x, 'PAR_CARRIERE_Q')
test_x =my_drop(test_x, 'pAR_JOC_RAPPORT_GAGNANT_M')
test_x =my_drop(test_x, 'pAR_JOC_REUSSITE_GAGNE')
test_x =my_drop(test_x, 'pAR_JOC_VICTOIRE')
test_x =my_drop(test_x, 'PAR_PLACE')
test_x =my_drop(test_x, 'PAR_PLACE_Q')
test_x =my_drop(test_x, 'PAR_CLASSE_AGE')
test_x =my_drop(test_x, 'pAR_JOC_VICTOIRE')
test_x =my_drop(test_x, 'PAR_REUSSITE_3P')
test_x =my_drop(test_x, 'PAR_JOC_REU_PLACE')
test_x =my_drop(test_x, 'PAR_NUM')
test_x =my_drop(test_x, 'PAR_COTEDER')
test_x =my_drop(test_x, 'CORDE')
test_x =my_drop(test_x, 'musique')
test_x =my_drop(test_x, 'CHEVAL')
test_x =my_drop(test_x, 'Nb_partant')

test_x =my_drop(test_x, 'autostart')
test_x =my_drop(test_x, 'grande_piste')
test_x =my_drop(test_x, 'cendre')

test_x =my_drop(test_x, 'PAR_PROPRIO')
test_x =my_drop(test_x, 'NOM_JOC')
test_x =my_drop(test_x, 'NOM_ENTR')

test_x =my_drop(test_x, 'HIPPO')
test_x =my_drop(test_x, 'PAR_AGE')
test_x =my_drop(test_x, 'POIDS')
test_x =my_drop(test_x, 'CO_DISTANCE')
test_x =my_drop(test_x, 'CO_PRIX')
test_x =my_drop(test_x, 'PAR_GAIN_NORMA')
test_x =my_drop(test_x, 'CHEVAL2')
test_x =my_drop(test_x, 'PAR_REUSSITE_3P2')
test_x =my_drop(test_x, 'PAR_REUSSITE_QUINTE2')
test_x =my_drop(test_x, 'PAR_CLASSE_AGE2')
test_x =my_drop(test_x, 'PAR_COTEDER2')
test_x =my_drop(test_x, 'Point')

test_x =my_drop(test_x, 'MUSIC_CHEVAL')
test_x =my_drop(test_x, 'MUSIC_ENT')
test_x =my_drop(test_x, 'MUSIC_JOC')
test_x =my_drop(test_x, 'PAR_VALEUR')

test_x =my_drop(test_x, 'MY_REUSSITE_CHEVAL')
test_x =my_drop(test_x, 'MY_REUSSITE_JOC')

test_x =my_drop(test_x, 'MY_REUSSITE_ENT')
test_x =my_drop(test_x, 'MY_ECART_JOC')
test_x =my_drop(test_x, 'CLA_AGE_PRIX')
test_x =my_drop(test_x, 'MY_auto_start')


test_x =my_drop(test_x, 'PAR_ENT_ECART_PLACE')
test_x =my_drop(test_x, 'PAR_VICTOIRE')
test_x =my_drop(test_x, 'PAR_VICTOIRE_Q')
test_x =my_drop(test_x, 'CHEVAL_QUINTE')
test_x =my_drop(test_x, 'PAR_ENT_NB_COURSE')
test_x =my_drop(test_x, 'p2018')


test_x['v0'] = 0.0
test_x['v1'] = 0.0
test_x['sel'] = 0




test_x.head(10)

In [ ]:
# test_x
cumul = 1
nb_rows = len(df_proba.index)
print("nb_rows=", nb_rows)
print(" FORMATION DU FICHIER ...",allure_etudier)
for i in range(0, nb_rows):
        n = df_proba[0][i]
        test_x['v0'][i] = n
        
        n = df_proba[1][i]
        test_x['v1'][i] = n
        


timer(start_time)

In [ ]:
test_x.head(20)

In [ ]:
print("\n\n fin de copie sur Test_x", test_x.head(10))

if (allure_etudier == 1):
    test_x.to_csv("d:\py_resultat_trot.csv")

if (allure_etudier == 2):
    test_x.to_csv("d:\py_resultat_galop.csv")

if (allure_etudier == 3):
    test_x.to_csv("d:\py_resultat_trot_monte.csv")

if (allure_etudier == 4):
    test_x.to_csv("d:\py_resultat_haie.csv")

if (allure_etudier == 5):
    test_x.to_csv("d:\py_resultat_steeple.csv")
